# Airport Lounge Access Agent
## Overview

In this example we will guide you through how to create a Strands Agent for automated lounge access. We will build a lounge access assistant connecting to an [Amazon Bedrock Knowledge Base](https://aws.amazon.com/bedrock/knowledge-bases/) and use tools to mock interactions with a database to store customer details such as frequent flier miles. We will use [Automated Reasoning checks from Amazon Bedrock Guardrails](https://docs.aws.amazon.com/bedrock/latest/userguide/guardrails-automated-reasoning-checks.html) to validate claims made by the lounge access assistant. 

## Agent Details
<div style=\"float: left; margin-right: 20px;\">
    

|Feature             |Description                                        |
|--------------------|---------------------------------------------------|
|Native tools used   |current_time, retrieve                             |
|Custom tools created|lookup_passenger_by_ff_number, lookup_passenger_by_name, check_star_alliance_gold_status, check_paid_lounge_membership, get_passenger_flight_history, calculate_tier_qualification_progress, validate_lounge_access_eligibility|
|Agent Structure     |Single agent architecture                          |
|AWS services used   |Amazon Bedrock Knowledge Base, Automated Reasoning checks from Amazon Bedrock Guardrails     |
</div>

In [ ]:
# installing pre-requisites
!pip install -r requirements.txt

In [ ]:
import os
import sys
import re 
import json
import boto3
from strands import Agent, tool
from strands.models import BedrockModel
from typing import Dict, List, Any, Optional
from datetime import datetime
from strands.hooks import HookProvider, HookRegistry, MessageAddedEvent
from botocore.config import Config as BotocoreConfig
import logging

# Configure logging
logger = logging.getLogger(__name__)

# Get the Bedrock Agent Runtime client
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime')

In [ ]:
# Fill in the required values
KB_ID = "<KNOWLEDGE_BASE_ID>"                               # Replace with your Knowledge Base ID
GUARDRAIL_ID = "<GUARDRAIL_ID>"                             # Replace with your Guardrail ID
GUARDRAIL_VERSION = "<GUARDRAIL_VERSION>"                   # Replace with your Guardrail Version if different from the default
MODEL_ID = "us.anthropic.claude-sonnet-4-20250514-v1:0"     # Replace with another model if you choose to do so

In [ ]:
@tool
def retrieve_lounge_information(
    query: str,
    knowledge_base_id: str = KB_ID,
    max_results: int = 5
) -> Dict[str, Any]:
    """
    Retrieve information from the Bedrock Knowledge Base about airport lounges, policies, and procedures.
    
    This tool queries the comprehensive lounge documentation including access policies, amenities, 
    dining options, business facilities, and frequently asked questions.
    
    Args:
        query: Natural language query about lounge access, amenities, policies, or procedures
        knowledge_base_id: Knowledge Base ID to query (optional)  
        max_results: Maximum number of results to return (1-10, default: 5)
        
    Returns:
        Dictionary containing query results, metadata, and success status
        
    Examples:
        - retrieve_lounge_information("What are the lounge access rules for United First Class?")
        - retrieve_lounge_information("What dining options are available?", content_filter="amenity")
        - retrieve_lounge_information("Can I bring guests with Star Alliance Gold?", max_results=3)
    """
    try:
        # Validate inputs
        if not query or not query.strip():
            return {
                "success": False,
                "error": "Query parameter is required and cannot be empty",
                "results": [],
                "query": query
            }
        
        # Clamp max_results to valid range
        max_results = max(1, min(max_results, 10))
            
        # Execute knowledge base retrieval
        response = bedrock_agent_runtime.retrieve(
            knowledgeBaseId=knowledge_base_id,
            retrievalQuery={
                'text': query
            },
            retrievalConfiguration={
                'vectorSearchConfiguration': {
                    'numberOfResults': max_results,
                    'overrideSearchType': 'SEMANTIC'  # Use both semantic and keyword search
                }
            }
        )
        
        # Process and format results
        formatted_results = []
        for result in response.get('retrievalResults', []):
            content_text = result.get('content', {}).get('text', '')
            formatted_result = {
                "content": content_text,
                "relevance_score": result.get('score', 0),
                "metadata": result.get('metadata', {}),
                "location": result.get('location', {})
            }
            formatted_results.append(formatted_result)
        
        # Sort by relevance score (highest first)
        formatted_results.sort(key=lambda x: x.get('relevance_score', 0), reverse=True)
        
        return {
            "success": True,
            "query": query,
            "results": formatted_results,
            "total_results": len(formatted_results),
            "timestamp": datetime.now().isoformat(),
            "knowledge_base_id": knowledge_base_id
        }
        
    except Exception as e:
        logger.error(f"Error executing knowledge base query: {e}")
        return {
            "success": False,
            "error": f"Knowledge base query failed: {str(e)}",
            "results": [],
            "query": query,
            "timestamp": datetime.now().isoformat()
        }


In [ ]:
# Test Knowledge Base
retrieve_lounge_information('What time is breakfast available in the lounges?')

In [ ]:
bedrock_model = BedrockModel(
    model_id=MODEL_ID,
    max_tokens=64000,
    # boto_client_config=Config(
    #    read_timeout=900,
    #    connect_timeout=900,
    #    retries=dict(max_attempts=3, mode="adaptive"),
    # ),
    additional_request_fields={
        "thinking": {
            "type": "disabled",
            # "budget_tokens": 2048,
        }
    },
)

In [ ]:
# Test function to demonstrate all tools

from passenger_data_tools import (
    lookup_passenger_by_ff_number,
    lookup_passenger_by_name,
    check_star_alliance_gold_status,
    check_paid_lounge_membership,
    get_passenger_flight_history,
    calculate_tier_qualification_progress,
    validate_lounge_access_eligibility
)

def test_passenger_tools():
    """Test all passenger data tools with sample data"""
    print("Testing Passenger Data Tools")
    print("=" * 50)
    
    test_ff_numbers = ["UA123456789", "LH987654321"]
    
    for ff_number in test_ff_numbers:
        print(f"\nTesting with FF Number: {ff_number}")
        print("-" * 30)
        
        # Test lookup
        lookup_result = lookup_passenger_by_ff_number(ff_number)
        print(f"Lookup: {'Found' if lookup_result.get('found') else 'Not Found'}")
        
        if lookup_result.get('found'):
            # Test Gold status
            gold_result = check_star_alliance_gold_status(ff_number)
            print(f"Gold Status: {gold_result.get('has_status')}")

            # Test lounge eligibility
            eligibility = validate_lounge_access_eligibility(ff_number, "First", True, "United")
            print(f"Lounge Eligible: {eligibility.get('eligible')} - {eligibility.get('reason', 'N/A')}")


test_passenger_tools()

In [ ]:
# System prompt for lounge access agent
system_prompt = """
You are a professional Airport Lounge Access Agent. You help passengers determine their 
lounge access eligibility and provide information about lounge amenities and policies.

IMPORTANT TOOLS AVAILABLE:
- lookup_passenger_by_ff_number: Look up passenger by frequent flier number
- lookup_passenger_by_name: Look up passenger by name  
- check_star_alliance_gold_status: Check Gold status and benefits
- check_paid_lounge_membership: Check paid lounge memberships
- validate_lounge_access_eligibility: Comprehensive eligibility check
- get_passenger_flight_history: Get recent flight history
- calculate_tier_qualification_progress: Check tier progress
- retrieve_lounge_information: Query lounge knowledge base

USAGE GUIDELINES:
1. Always use tools to look up passenger data when provided with FF numbers or names
2. Use validate_lounge_access_eligibility for comprehensive lounge access decisions
3. Use retrieve_lounge_information to answer questions about lounge amenities, policies, or procedures
4. Be helpful, accurate, and professional in all responses
5. Clearly explain lounge access rules and any restrictions
"""


In [ ]:
# Create agent with tools
agent = Agent(
    system_prompt=system_prompt,
    model=bedrock_model,
    tools=[
        lookup_passenger_by_ff_number,
        lookup_passenger_by_name,
        check_star_alliance_gold_status,
        check_paid_lounge_membership,
        validate_lounge_access_eligibility,
        get_passenger_flight_history,
        calculate_tier_qualification_progress,
        retrieve_lounge_information
    ]
)

In [ ]:
# Test queries
test_queries = [
    "Check lounge access for frequent flier number UA123456789 flying First Class internationally"]
    # "What dining options are available in the lounge?",
    # "Look up passenger LH987654321 and check their Gold status",
    # "Can United domestic First Class passengers access the lounge?",
    # "What are the lounge operating hours?"


for i, query in enumerate(test_queries, 1):
    print(f"\nTest Query {i}: {query}")
    print("-" * 40)
    
    try:
        response = agent(query)
        print(f"Response: {response}")
        
    except Exception as e:
        print(f"Error processing query: {e}")
        
print("\n" + "=" * 50)
print("Strands agent testing completed")

### Adding Automated Reasoning checks Guardrail to the Strands agent

In [ ]:
class EnhancedGuardrailsHook(HookProvider):
    """
    Enhanced Guardrail Hook that:
    1. Only triggers on agent responses that include validate_lounge_access_eligibility tool usage
    2. Includes passenger context extracted from tool results
    3. Passes comprehensive passenger variables to the Automated Reasoning policy
    """
    
    def __init__(self, guardrail_id: str, guardrail_version: str):
        self.guardrail_id = guardrail_id
        self.guardrail_version = guardrail_version
        self.bedrock_client = boto3.client("bedrock-runtime")
        
        # Store message history and tool usage tracking
        self.message_history = []
        self.recent_tool_results = {}
        self.validate_eligibility_used = False
        
    def register_hooks(self, registry: HookRegistry) -> None:
        registry.add_callback(MessageAddedEvent, self.process_message)
    
    def detect_validate_eligibility_tool_usage(self, content: str) -> bool:
        """
        Detect if validate_lounge_access_eligibility tool was used in the message
        """
        # Look for the tool call pattern in the message content
        tool_patterns = [
            r"validate_lounge_access_eligibility",
            r"validate.*eligibility", 
            r"Comprehensive lounge access eligibility validation",
            r'"tool_name":\s*"validate_lounge_access_eligibility"',
            r"I'll check your lounge access eligibility",
            r"checking.*eligibility",
            r"eligibility.*validated",
            r"access eligibility.*check"
        ]
        
        content_lower = content.lower()
        return any(re.search(pattern, content_lower) for pattern in tool_patterns)
    
    def extract_passenger_context_from_tool_results(self) -> Dict[str, Any]:
        """
        Extract passenger context from validate_lounge_access_eligibility tool results
        and message history
        """
        context = {
            "passenger_details": {},
            "flight_details": {},
            "eligibility_factors": {},
            "tool_results": self.recent_tool_results,
            "conversation_context": []
        }
        
        # Extract from recent tool results if available
        if "validate_lounge_access_eligibility" in self.recent_tool_results:
            tool_result = self.recent_tool_results["validate_lounge_access_eligibility"]
            
            # Extract passenger data from tool result
            passenger_data = tool_result.get("passenger_data", {})
            if passenger_data:
                context["passenger_details"]["frequent_flier_number"] = passenger_data.get("frequent_flier_number", "")
                context["passenger_details"]["name"] = passenger_data.get("name", "")
                context["passenger_details"]["tier_status"] = passenger_data.get("tier_status", "")
                context["passenger_details"]["airline"] = passenger_data.get("airline", "")
                context["passenger_details"]["total_miles"] = passenger_data.get("total_miles", 0)
            
            # Extract Gold status info
            gold_status = tool_result.get("gold_status", {})
            if gold_status:
                context["eligibility_factors"]["gold_status"] = gold_status.get("has_status", False)
                context["eligibility_factors"]["status_level"] = gold_status.get("status_level", "")
                context["eligibility_factors"]["companion_passes"] = gold_status.get("companion_passes", 0)
            
            # Extract eligibility decision
            context["eligibility_factors"]["eligible"] = tool_result.get("eligible", False)
            context["eligibility_factors"]["access_type"] = tool_result.get("access_type", "")
            context["eligibility_factors"]["reason"] = tool_result.get("reason", "")
        
        # Extract additional information from message history
        for message in self.message_history:
            content = message.get('content', '')
            role = message.get('role', '')
            
            # Extract frequent flier numbers
            ff_numbers = re.findall(r'\b([A-Z]{2}\d{8,9})\b', content)
            if ff_numbers and not context["passenger_details"].get("frequent_flier_numbers"):
                context["passenger_details"]["frequent_flier_numbers"] = ff_numbers
            
            # Extract flight information
            flight_numbers = re.findall(r'\b([A-Z]{2}\d{3,4})\b', content)
            if flight_numbers:
                context["flight_details"]["flight_numbers"] = flight_numbers
            
            # Extract class of service mentions
            class_mentions = re.findall(r'\b(First Class|Business Class|Premium Economy|Economy)\b', content, re.IGNORECASE)
            if class_mentions:
                context["flight_details"]["class_mentions"] = class_mentions
            
            # Store conversation context (limited)
            context["conversation_context"].append({
                "role": role,
                "content": content[:150],  # First 150 chars
                "timestamp": datetime.now().isoformat()
            })
        
        return context
    
    def extract_policy_variables_from_context(self, passenger_context: Dict[str, Any], message_content: str) -> Dict[str, Any]:
        """
        Extract variables needed for the Automated Reasoning policy from passenger context
        """
        variables = {
            # Passenger identification
            "passenger_frequent_flier_numbers": passenger_context.get("passenger_details", {}).get("frequent_flier_numbers", []),
            "passenger_names": passenger_context.get("passenger_details", {}).get("names", []),
            
            # Flight details
            "flight_numbers": passenger_context.get("flight_details", {}).get("flight_numbers", []),
            "class_of_service": passenger_context.get("flight_details", {}).get("class_mentions", []),
            
            # Status and eligibility factors
            "membership_status": passenger_context.get("eligibility_factors", {}).get("status_mentions", []),
            "access_decision_terms": passenger_context.get("eligibility_factors", {}).get("access_mentions", []),
            
            # Message analysis
            "response_contains_eligibility": self.detect_validate_eligibility_tool_usage(message_content),
            "message_length": len(message_content),
            "timestamp": datetime.now().isoformat(),
            
            # Extracted policy-relevant information
            "mentioned_lounges": [],
            "mentioned_airlines": [],
            "mentioned_airports": []
        }
        
        # Extract lounge names
        lounge_patterns = [
            r"\b(United Club|Star Alliance|Polaris|Lufthansa|Air Canada Maple Leaf|British Airways)\s*(?:Lounge|Club)\b",
            r"\b([A-Z][a-z]+\s+(?:Lounge|Club))\b"
        ]
        for pattern in lounge_patterns:
            matches = re.findall(pattern, message_content, re.IGNORECASE)
            variables["mentioned_lounges"].extend(matches)
        
        # Extract airline names
        airline_pattern = r"\b(United Airlines|Lufthansa|Air Canada|British Airways|Singapore Airlines|Thai Airways)\b"
        airline_matches = re.findall(airline_pattern, message_content, re.IGNORECASE)
        variables["mentioned_airlines"] = airline_matches
        
        # Extract airport codes
        airport_pattern = r"\b([A-Z]{3})\b"
        airport_matches = re.findall(airport_pattern, message_content)
        variables["mentioned_airports"] = airport_matches
        
        return variables
    
    def evaluate_content_with_context(self, content: str, source: str = "INPUT"):
        """
        Enhanced content evaluation that only triggers guardrails when validate_lounge_access_eligibility tool is used
        """
        print(f"[ENHANCED_GUARDRAIL] [evaluate_content]: source: {source}, content length: {len(content)}")
        
        if source == "INPUT":
            # Store input for context and reset tool usage tracking
            self.message_history.append({"role": "user", "content": content})
            self.validate_eligibility_used = False
            self.recent_tool_results = {}
            return
        
        # Only process OUTPUT messages when validate_eligibility_used flag is set
        if source == "OUTPUT" and self.validate_eligibility_used:
            print(f"[ENHANCED_GUARDRAIL] Tool usage flag set, applying guardrail validation...")
            
            # Extract tool results from message content (simplified for this implementation)
            # In a real implementation, this would capture actual tool execution results
            self._extract_tool_results_from_content(content)
            
            # Extract comprehensive passenger context from tool results and history
            passenger_context = self.extract_passenger_context_from_tool_results()
            
            # Extract policy variables
            policy_variables = self.extract_policy_variables_from_context(passenger_context, content)
            
            # Get the most recent user input for context
            recent_input = ""
            for msg in reversed(self.message_history):
                if msg.get("role") == "user":
                    recent_input = msg.get("content", "")
                    break
            
            # Create enriched query with passenger context embedded
            enriched_query = f"""User Query: {recent_input}

Passenger Context for Policy Evaluation:
{json.dumps(passenger_context, indent=2)}

Policy Variables Extracted:
{json.dumps(policy_variables, indent=2)}

Tool Used: validate_lounge_access_eligibility"""

            # Create content for guardrail validation using only valid qualifiers
            content_to_validate = [
                {
                    "text": {
                        "text": enriched_query, 
                        "qualifiers": ["query"]
                    }
                },
                {
                    "text": {
                        "text": content, 
                        "qualifiers": ["guard_content"]
                    }
                }
            ]
            
            print(f"[ENHANCED_GUARDRAIL] Applying guardrail with embedded passenger context:")
            print(f"[ENHANCED_GUARDRAIL] - Tool detected: validate_lounge_access_eligibility")
            print(f"[ENHANCED_GUARDRAIL] - Policy variables: {len(policy_variables)} items")
            print(f"[ENHANCED_GUARDRAIL] - Passenger context: {len(passenger_context)} sections")
            print(f"[ENHANCED_GUARDRAIL] - Enriched query length: {len(enriched_query)} chars")
            
            try:
                response = self.bedrock_client.apply_guardrail(
                    guardrailIdentifier=self.guardrail_id,
                    guardrailVersion=self.guardrail_version,
                    source="OUTPUT",
                    content=content_to_validate
                )
                
                print(f"[ENHANCED_GUARDRAIL] Guardrail response: {response.get('action', 'UNKNOWN')} - {response.get('actionReason', 'No reason provided')}")
                
                # Print the complete guardrail response for analysis
                print(f"[ENHANCED_GUARDRAIL] ========== COMPLETE GUARDRAIL RESPONSE ==========")
                print(json.dumps(response, indent=2, default=str))
                print(f"[ENHANCED_GUARDRAIL] ================================================")
                
                # Specifically extract and display automatedReasoningPolicy findings
                if response.get('assessments'):
                    print(f"[ENHANCED_GUARDRAIL] ========== AUTOMATED REASONING FINDINGS ==========")
                    for i, assessment in enumerate(response.get('assessments', [])):
                        print(f"[ENHANCED_GUARDRAIL] Assessment {i + 1}:")
                        
                        # Look for automatedReasoningPolicy findings
                        if 'automatedReasoningPolicy' in assessment:
                            ar_policy = assessment['automatedReasoningPolicy']
                            print(f"[ENHANCED_GUARDRAIL] Automated Reasoning Policy Results:")
                            print(json.dumps(ar_policy, indent=4, default=str))
                        
                        # Log processing metrics
                        if assessment.get('invocationMetrics'):
                            metrics = assessment['invocationMetrics']
                            print(f"[ENHANCED_GUARDRAIL] Processing latency: {metrics.get('guardrailProcessingLatency', 0)}ms")
                    
                    print(f"[ENHANCED_GUARDRAIL] ==============================================")
                else:
                    print(f"[ENHANCED_GUARDRAIL] No assessments found in guardrail response")
                
                return response
                
            except Exception as e:
                print(f"[ENHANCED_GUARDRAIL] Error applying guardrail: {str(e)}")
                return None
        else:
            # For messages without validate_lounge_access_eligibility tool usage, just log
            print(f"[ENHANCED_GUARDRAIL] No validate_lounge_access_eligibility tool usage detected, skipping guardrail validation")
            return None
    
    def _extract_tool_results_from_content(self, content: str):
        """
        Extract tool results from message content (simplified implementation)
        In a real implementation, this would hook into the actual tool execution
        """
        # Look for patterns that indicate tool results in the content
        if "eligible" in content.lower() and ("lounge access" in content.lower() or "access" in content.lower()):
            # Mock tool result for demonstration
            self.recent_tool_results["validate_lounge_access_eligibility"] = {
                "eligible": "eligible" in content.lower() and "not eligible" not in content.lower(),
                "reason": "Extracted from content analysis",
                "access_type": "Detected from response",
                "passenger_data": {},
                "gold_status": {},
                "paid_membership": {}
            }
    
    def process_message(self, event: MessageAddedEvent) -> None:
        """Process messages and maintain context - FIXED VERSION"""
        print(f'[ENHANCED_GUARDRAIL] [process_message] Processing message from: {event.message.get("role")}')
        
        if event.message.get("role") == "user":
            content = "".join(block.get("text", "") for block in event.message.get("content", []))
            if content:
                self.evaluate_content_with_context(content, "INPUT")
        
        elif event.message.get("role") == "assistant":
            # Extract text content
            content = "".join(block.get("text", "") for block in event.message.get("content", []))
            
            # Check for tool usage in the message content blocks
            tool_used = False
            for block in event.message.get("content", []):
                if "toolUse" in block:
                    tool_name = block["toolUse"].get("name", "")
                    print(f"[ENHANCED_GUARDRAIL] Found tool in message: {tool_name}")
                    if tool_name == "validate_lounge_access_eligibility":
                        tool_used = True
                        print(f"[ENHANCED_GUARDRAIL] ✓ Detected validate_lounge_access_eligibility tool usage!")
                        # Store the tool input for context
                        self.recent_tool_results["validate_lounge_access_eligibility"] = {
                            "tool_input": block["toolUse"].get("input", {}),
                            "tool_use_id": block["toolUse"].get("toolUseId", "")
                        }
                        # Mark that we're waiting for the tool result
                        self.validate_eligibility_used = True
                        break
            
            if content:
                # Add to message history for context
                self.message_history.append({"role": "assistant", "content": content, "tool_used": tool_used})
                
                # Evaluate if tool was used OR if we're processing the result of a previous tool call
                if tool_used or self.validate_eligibility_used:
                    print(f"[ENHANCED_GUARDRAIL] Triggering guardrail validation (tool_used={tool_used}, waiting_for_result={self.validate_eligibility_used})")
                    self.evaluate_content_with_context(content, "OUTPUT")
                    # Reset flag after processing the final response
                    if not tool_used and self.validate_eligibility_used:
                        self.validate_eligibility_used = False
                else:
                    print(f"[ENHANCED_GUARDRAIL] No validate_lounge_access_eligibility tool usage detected, skipping guardrail validation")

In [ ]:
# Create agent with enhanced guardrail hook
agent_enhanced_guardrail = Agent(
    system_prompt=system_prompt,
    model=bedrock_model,
    tools=[
        lookup_passenger_by_ff_number,
        lookup_passenger_by_name,
        check_star_alliance_gold_status,
        check_paid_lounge_membership,
        validate_lounge_access_eligibility,
        get_passenger_flight_history,
        calculate_tier_qualification_progress,
        retrieve_lounge_information
    ],
    hooks=[EnhancedGuardrailsHook(GUARDRAIL_ID, GUARDRAIL_VERSION)]
)

In [ ]:
# Test queries
test_queries = [
    "Check lounge access for frequent flier number UA123456789 flying First Class internationally"]
    # "What dining options are available in the lounge?",
    # "Look up passenger LH987654321 and check their Gold status",
    # "Can United domestic First Class passengers access the lounge?",
    # "What are the lounge operating hours?"


for i, query in enumerate(test_queries, 1):
    print(f"\n{'='*60}")
    print(f"Test Query {i}: {query}")
    print('='*60)
    
    try:
        response = agent_enhanced_guardrail(query)
        print(f"Response: {response}")
        
    except Exception as e:
        print(f"Error processing query: {e}")